# Section validé

In [ ]:
#!kill -9 -1

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Import de spacy, librairies de traitement de texte. Liens utiles : 

*  https://medium.com/@dhifallah.othmen/extraction-et-entra%C3%AEnement-des-entit%C3%A9s-nomm%C3%A9es-avec-spacy-90f7e1a219d7
*  https://course.spacy.io/fr/
*  https://datacorner.fr/spacy/

In [ ]:
!pip install -U spacy
!python -m spacy download fr_core_news_md
import spacy
from spacy import displacy

nlp = spacy.load('fr_core_news_md')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
2023-04-25 08:03:22.300357: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-25 08:03:23.246342: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-04-25 08:03:24.715101: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-04-25 08:03:24.715689: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node rea

Code qui permet la normalisation du texte

In [ ]:
import re #Pour le traitement de HTML
import numpy as np #Import par défaut
from bs4 import BeautifulSoup #Pour extraire plus facilement de l'html
from collections import Counter #Pour la sicion du jeux de DD

In [ ]:
# Charger le fichier HTML
with open("/content/drive/MyDrive/Colab Notebooks/gd-fiscalite_depenses-utf8-sgml-cwb.html", "r") as f:
    html = f.read()

# Supprimer toutes les balises HTML
html = re.sub(r'<[^>]*?>', '', html)

# Écrire le contenu modifié dans un fichier HTML
with open("fichier_modifie.html", "w") as f:
    f.write(html)

# Supprimer les lignes vides du fichier modifié
with open("fichier_modifie.html", "r") as f:
    lines = f.readlines()

lines = [line.strip() for line in lines if line.strip()]

with open("fichier_sans_lignes_vides.html", "w") as f:
    f.write("\n".join(lines))

Affiche le début du contenu du texte "fichier_sans_lignes_vides.html"

In [ ]:
from IPython.display import HTML

with open("fichier_sans_lignes_vides.html", "r") as f:
    content = f.readlines()
    
HTML("".join(content[:10]))#Affiche les 10 premieres lignes

Premier traitement de texte avec spacy tokenisation de tout le texte

In [ ]:
def process_doc(doc):
    word_freq = Counter()
    for token in doc:
        if not token.is_stop and not token.is_punct and not token.is_space:
            word_freq[token.text] += 1
    return word_freq

# Ouvrir le fichier et récupérer son contenu
with open("fichier_sans_lignes_vides.html", "r", encoding="utf-8") as f:
    lines = f.readlines()

# Nombre de lignes à traiter à la fois
chunk_size = 1000

# Parcourir le texte en segments de taille chunk_size
word_freq = Counter()
for i in range(0, len(lines), chunk_size):
    chunk = "".join(lines[i:i+chunk_size])
    doc = nlp(chunk)
    chunk_word_freq = process_doc(doc)
    word_freq += chunk_word_freq

# Écrire les résultats dans un fichier texte
with open("Top mots.txt", "w", encoding="utf-8") as f:
    for word, freq in word_freq.most_common():
        f.write(f"{word}\t{freq}\n")


# Section en test 

In [ ]:
text = "PokémonN 1 (prononcé [pɔ.ke.mɔn], connue au Japon sous le nom de Pocket MonstersN 2) est une franchise créée par Satoshi Tajiri en 1996, présente en particulier en jeu vidéo, dans des séries éditées par Nintendo. Selon les statistiques de Nintendo en 2010, les jeux Pokémon se sont vendus à environ 250 millions d’unités. Le jeu vidéo Pokémon Rouge et Bleu s’est vendu à plus de 30 millions d’exemplaires, ce qui en fait un record des ventes dans l’histoire du jeu vidéo. La franchise est également exploitée sous forme d'anime, de mangas, et de jeux de cartes à collectionner. Dans la série animée homonyme, le personnage principal, Sacha, voyage à travers diverses régions fictives dans le but d’attraper de nouvelles sortes de monstres éponymes, un concept qu’on retrouve également dans les jeux vidéo de la franchise. Pokémon a eu un impact culturel très important dans les pays où il a été introduit, dont le Japon, les États-Unis, le Canada, la France et d'autres pays européens."

In [ ]:
from textblob import TextBlob
doc = nlp(text)

# Utilisation de TextBlob pour effectuer une analyse de sentiment
blob = TextBlob(text)
polarity, subjectivity = blob.sentiment

print("Polarité du texte:", polarity)
print("Subjectivité du texte:", subjectivity)


NameError: ignored

In [ ]:
import spacy
import gensim
from gensim.corpora import Dictionary
from gensim.models.ldamodel import LdaModel
from gensim.models.coherencemodel import CoherenceModel

# Exemple de texte
text = """
Le basketball est un sport collectif qui se joue avec une balle. Il oppose deux équipes de cinq joueurs 
chacune qui se relaient en défense et en attaque. Le but est de marquer des points en lançant la balle 
dans un panier placé à plusieurs mètres de hauteur. Le basketball est très populaire dans le monde entier, 
et de nombreuses compétitions sont organisées chaque année.
"""

# Prétraitement du texte
doc = nlp(text)
sentences = [[token.lemma_ for token in sentence if not token.is_stop and token.is_alpha] for sentence in doc.sents]

# Création du dictionnaire
dictionary = Dictionary(sentences)
corpus = [dictionary.doc2bow(sentence) for sentence in sentences]

# Entraînement du modèle LDA
lda_model = LdaModel(corpus, num_topics=3, id2word=dictionary, passes=10, random_state=42)

# Calcul de la cohérence thématique
coherence_model_lda = CoherenceModel(model=lda_model, texts=sentences, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()

print("Score de cohérence thématique : ", coherence_lda)


Score de cohérence thématique :  0.3954497310087454


Voici quelques idées supplémentaires pour analyser votre texte avec SpaCy :

Analyse de la syntaxe : Utilisez SpaCy pour analyser la syntaxe de votre texte. Vous pouvez identifier les parties du discours, les relations entre les mots, les phrases nominales, les phrases verbales, etc. Cela peut vous aider à comprendre comment le texte est structuré et comment les idées sont présentées.

Analyse des entités nommées : Utilisez SpaCy pour extraire les entités nommées du texte. Cela peut vous aider à identifier les personnes, les lieux, les organisations, les dates, les montants d'argent, etc. que le texte mentionne. Vous pouvez également utiliser cette information pour créer une visualisation de réseau montrant les relations entre les entités nommées.

Analyse des sentiments : Utilisez SpaCy pour effectuer une analyse des sentiments sur le texte. Cela peut vous aider à comprendre si le texte est positif, négatif ou neutre. Vous pouvez également identifier les mots ou les phrases qui contribuent le plus à l'orientation sentimentale du texte.

Analyse de similarité : Utilisez SpaCy pour calculer la similarité entre les phrases du texte. Cela peut vous aider à identifier les sections du texte qui traitent des mêmes sujets ou qui ont des idées similaires.

Analyse de la cohérence thématique : Utilisez SpaCy pour identifier les thèmes principaux du texte. Cela peut vous aider à comprendre les idées principales du texte et comment elles sont développées. Vous pouvez également utiliser cette information pour créer une visualisation de nuage de mots montrant les mots les plus fréquemment utilisés dans le texte.

In [ ]:
with open('/content/drive/MyDrive/Colab Notebooks/31.txt', 'r') as fichier:
    text = fichier.read()

print(text)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



# Début du traitement

Lecture du fichier **occitanie.txt** et intégration des données dans la variables **Dep**

In [ ]:
import re

# Charger le fichier occitanie.txt
with open("occitanie.txt", "r", encoding="utf-8") as file:
    lines = file.readlines()

# Initialiser le dictionnaire Dep
Dep = {}

# Traiter chaque ligne du fichier
for line in lines:
    # Rechercher le code départemental et le texte associé
    match = re.match(r"FR-(\d+):\[(.+)\]", line.strip())

    if match:
        # Extraire le code départemental et le texte
        dep_code = int(match.group(1))
        dep_text = match.group(2)

        # Nettoyer le texte en supprimant les guillemets et les espaces superflus
        cleaned_text = dep_text.replace('"', "").strip()

        # Ajouter l'entrée dans le dictionnaire Dep
        Dep[dep_code] = cleaned_text


Vérification du bon fonctionement

In [ ]:
print(len(Dep)) #Nombre de valeur
dept_num, dept_text = list(Dep.items())[5] #Récup le num et le contenue
print(f"Département {dept_num} : {dept_text}") #Affiche

13
Département 11 : Affichages municipaux. Messages télévisuels. Expliquer aux citoyens l'utilisation de l'argent public., 'Tout le monde devrait participer, les plus démunis pourrait participer par des heures de participation à la vie municipale.', 'Tous les citoyens devrait pouvoir vivre dignement: être logés et avoir à manger.', 'Un toit pour tous. Ne pas attendre après des bénévoles pour nourrir la population', 'Les gouvernants doivent rendre des comptes sur leurs dépenses.', 'Un toit et à manger pour tous.', 'Aucune', 'Baisser tous les impôts surtout pour les classes moyennes qui sont les grands oubliés comme d habitude..', 'Impôts sur le revenu, impôts fonciers. Et mettre fin a la taxe d habitation dès a présent.', 'Aucune. Marre de tout financer.', 'Élargir la prime d activité aux classes moyennes. Concernant les APL également augmenter le plafond. Augmenter la prime versée pour les parents qui ont a chargent des enfants de moins de 3 ans. Participation a la prise en charge de l

In [ ]:
import os
import string
from collections import Counter
import spacy

# fonction pour traiter un texte et renvoyer un objet Counter contenant la fréquence de chaque mot
def process_text(text):
    word_freq = Counter()
    max_length = 1000000
    start = 0
    end = max_length

    while start < len(text):
        doc = nlp(text[start:end])
        for token in doc:
            if not token.is_stop and not token.is_punct and not token.is_space:
                word_freq[token.text] += 1

        start += max_length
        end += max_length

    return word_freq

# parcourir le dictionnaire et compter la fréquence de chaque mot dans chaque valeur
for dept_num, dept_text in Dep.items():
    word_freq = process_text(dept_text)

    # créer un nom de fichier pour le département
    file_name = f"Top mots_{dept_num}.txt"

    # écrire les résultats dans un fichier texte pour le département
    with open(file_name, "w", encoding="utf-8") as f:
        for word, freq in word_freq.most_common():
            f.write(f"{word}\t{freq}\n")
